# __init__

In [87]:
from openai import OpenAI
import openai
from dotenv import load_dotenv
import os
from tqdm import tqdm
import json
import time

import pandas as pd
import numpy as np
import ast
import re

from datetime import datetime
import pandas as pd
import numpy as np
from word2number import w2n
import re
import numpy as np

# Load environment variables from .env file in current directory
load_dotenv()

True

# Point to API Key

In [88]:
api_key = os.getenv("API_KEY")

In [89]:
if api_key:
    client = OpenAI(api_key=api_key)
else:
    print("API key not found. Please set your API key as an environment variable.")

# Load data (openai - responses)

In [90]:
df = pd.read_csv('data_extracted_llm_openai.csv')

# Functions

In [ ]:
## Clean up functions

In [91]:
def get_only_reasontimevehicle(d):
    filtered_dict = {}
    keys_to_keep = ['time_of_day', 'reason', 'vehicles']

    for key in keys_to_keep:
        if key in d:
            filtered_dict[key] = d[key]

    return filtered_dict

def dict_to_string(d):
    string_representation = '{'
    for key, value in d.items():
        if isinstance(value, str):
            string_representation += f"'{key}': '{value}', "
        else:
            string_representation += f"'{key}': {value}, "
    string_representation = string_representation.rstrip(', ') + '}'
    return string_representation


# Function to convert JSON string to dictionary
def json_str_to_dict(json_str):
    try:
        json_str = json_str.replace('output:', '')
        return ast.literal_eval(json_str)
    except:
        return "error:incorrect_format"

# LLM classification
[time_of_day,reason,vehicles]

In [96]:
class MessageInitializer:
    def __init__(self,option):
        if option == 1:
                self.message_history = [
            {
              "role": "system",
              "content":'''You will extract information from a news article and give me the output in json format.'''
            },
            {
              "role": "user",
              "content": "'New Delhi: Two students were injured after a speeding car rammed into the footpath they were walking on in central Delhi’s Ranjit Nagar. The errant driver has been booked, police said. The accident took place in the early hours of Tuesday and the students, in their early 20s, were discharged after treatment, a police officer said. “The driver who received some injuries was also rushed to the hospital. A case under sections 279 (rash driving), 337 (hurt due to negligence) and 338 (Punishment for grievous hurt) has been registered,” said the officer. The driver has been identified and the offending vehicle has been seized, police said. The purported CCTV footage of the incident started making rounds on social media on Thursday. In the video, a man and a woman can be seen walking on the footpath when a speeding car banged into the duo walking on the footpath. Explore Your Financial Landscape with Personalized Credit Insights.'"
            },
            {
              "role": "assistant",
              "content": "output:{\"fatalities\": 0,\"injured\":2,\"victim_gender\":[\"Male\",\"Female\"],\"vehicles\":[\"truck\",\"car\"],\"child_involved\":0,\"names_ages\":[(\"unknown_name\",20),(\"unknown_name\",20)]}/"
            }
          ]
        if option == 2:
                self.message_history = [
        {
          "role": "system",
          "content": "Input is a dict type.\n\nFor key value \"time_of_day\" classify and replace individual value to the following categories ['morning','afternoon','evening','night']\n\nFor key value \"reason\" classify and replace individual value to the following categories ['Speed','Driving Under Influence','Collision','Visibility','Negligence','External influences']\n\nFor key value \"vehicles\" classify and replace individual value to the following categories ['Car','Two-Wheeler','Commercial Vehicle','Special Vehicle']"
        },
        {
          "role": "user",
          "content": "'{\"time_of_day\":\"early morning\",\"reason\":\"rash driving\",\"vehicles\":[\"car\",\"tractor-trolly\"]}'"
        },
        {
          "role": "assistant",
          "content": "'{\"time_of_day\":\"morning\",\"reason\":\"Negligence\",\"vehicles\":[\"car\",\"'Special Vehicle'\"]}'"
        },
        {
          "role": "user",
          "content": "'{\"time_of_day\":\"early morning\",\"reason\":\"speeding\",\"vehicles\":[\"car\"]}'"
        },
        {
          "role": "assistant",
          "content": "'{\"time_of_day\":\"morning\",\"reason\":\"Speed\",\"vehicles\":[\"Car\"]}'"
        },
        {
          "role": "user",
          "content": "'{\"time_of_day\":\"wee hours\",\"reason\":\"Lost control over vehicle\",\"vehicles\":[\"BMW\",\"Truck\",\"scooty\"]}'"
        },
        {
          "role": "assistant",
          "content": "'{\"time_of_day\":\"night\",\"reason\":\"Negligence\",\"vehicles\":[\"Car\",\"Commercial Vehicle\",\"Two-Wheeler\"]}'"
        }
  ]
    
    def add_user_input(self, role, content):
        self.user_input = {"role": role, "content": content}
        self.prompt = self.message_history
        self.prompt.append(self.user_input)
        return (self.prompt)

# Applying cleanup functions 
[fatalities, injured, victim_gender]

In [95]:
df['fatalities'] = df['fatalities'].astype(str).apply(text_to_single_number)
df['injured'] = df['injured'].astype(str).apply(text_to_single_number)
df['victim_gender_t'] = df['victim_gender'].astype(str).apply(classify_gender)

# Individual test

In [97]:
# m = MessageInitializer(2)

# message = m.add_user_input("user","'{\"time_of_day\":\"night\",\"reason\":\"snorting drugs\",\"vehicles\":[\"Car\",\"Commercial Vehicle\",\"Two-Wheeler\"]}'")


response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages = message,
            temperature=0.5,
            max_tokens=256,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0
                                            )

response.choices[0].message.content

"{'time_of_day': 'night', 'reason': 'Collision', 'vehicles': ['Car', 'Special Vehicle', 'Commercial Vehicle', 'Two-Wheeler']}"

In [98]:
print(message)

[{'role': 'system', 'content': 'Input is a dict type.\n\nFor key value "time_of_day" classify and replace individual value to the following categories [\'morning\',\'afternoon\',\'evening\',\'night\']\n\nFor key value "reason" classify and replace individual value to the following categories [\'Speed\',\'Driving Under Influence\',\'Collision\',\'Visibility\',\'Negligence\',\'External influences\']\n\nFor key value "vehicles" classify and replace individual value to the following categories [\'Car\',\'Two-Wheeler\',\'Commercial Vehicle\',\'Special Vehicle\']'}, {'role': 'user', 'content': '\'{"time_of_day":"early morning","reason":"rash driving","vehicles":["car","tractor-trolly"]}\''}, {'role': 'assistant', 'content': '\'{"time_of_day":"morning","reason":"Negligence","vehicles":["car","\'Special Vehicle\'"]}\''}, {'role': 'user', 'content': '\'{"time_of_day":"early morning","reason":"speeding","vehicles":["car"]}\''}, {'role': 'assistant', 'content': '\'{"time_of_day":"morning","reason

# Preprocess the str for passing through the llm

Workflow: str --> dict(filter) --> str

In [99]:
df['JSON_Dict_filtered'] = df['JSON_Dict'].apply(json_str_to_dict)

In [100]:
df['JSON_Dict_filtered'] = df['JSON_Dict_filtered'].apply(get_only_reasontimevehicle)

In [101]:
df['JSON_Dict_filtered'] = df['JSON_Dict_filtered'].apply(dict_to_string)

In [104]:
# 

In [105]:
# %%time

print_outout = False

# --------------------------------------------
# --------------------------------------------

file_path = "classification_progress.json"

# Check if extraction_progress.json exists
if os.path.exists(file_path):
    with open(file_path, "r") as file:
        extraction_progress = json.load(file)
else:
    extraction_progress = {}
    with open(file_path, "w") as file:
        json.dump(extraction_progress, file)
        
# --------------------------------------------
# --------------------------------------------

# Filter out ids that have already been processed
processed_ids = list(map(int, extraction_progress.keys()))

remaining_ids = df[~df['id'].isin(processed_ids)]

count = 1

for index, row in tqdm(remaining_ids.iterrows(), total=remaining_ids.shape[0]):
        
    # ---message---
    m = MessageInitializer(2)
    message = m.add_user_input("user",row['JSON_Dict_filtered'])
    
    
    # ---send_message---
    try:
        response = client.chat.completions.create(
                    model="gpt-3.5-turbo-0125",
                    messages = message,
                    temperature=0.5,
                    max_tokens=256,
                    top_p=0.5,
                    frequency_penalty=0,
                    presence_penalty=0
                                                    )
    except:
        continue
        
    if print_outout == True:
        print('\n')
        print(m.user_input)
        print("-"*50)
        print(response.choices[0].message.content)
        print("-"*15)
        print("~"*5,"Output","~"*5)
        print('type -->',type(response.choices[0].message.content))
        print('total tokens -->',str(response.usage.total_tokens))
        print('prompt_tokens - ',response.usage.prompt_tokens,'response_tokens - ',response.usage.completion_tokens)
        print("--x"*15,'\n')
        
        
    try:
        extraction_progress[row['id']] = (response.choices[0].message.content)
    except:
        extraction_progress[row['id']] = 'error_openai'
        
    with open("classification_progress.json", "w") as file:
        json.dump(extraction_progress, file)
    
    count += 1
    
    if count % 2000 == 0:
        user_input = input("Continue (y/n)? ")
        if user_input.lower() != 'y':
            break
        
    
    
    time.sleep(0.2)
        

 23%|████████▌                            | 1998/8646 [30:45<1:36:20,  1.15it/s]

Continue (y/n)?  y


 46%|████████████████▏                  | 3998/8646 [1:03:14<1:13:27,  1.05it/s]

Continue (y/n)?  y


 69%|█████████████████████████▋           | 5998/8646 [1:36:49<45:11,  1.02s/it]

Continue (y/n)?  y


 93%|██████████████████████████████████▏  | 7998/8646 [2:10:52<14:40,  1.36s/it]

Continue (y/n)?  y


100%|█████████████████████████████████████| 8646/8646 [2:25:27<00:00,  1.01s/it]


In [ ]:
# This will extract from JSON 

In [106]:
import json
import pandas as pd
import os
import ast  # Add this import statement

# Check if extraction_progress.json exists
if os.path.exists("classification_progress.json"):
    with open("classification_progress.json", "r") as file:
        data = json.load(file)

    # Create DataFrame with keys and entire JSON string values
    df_json = pd.DataFrame(list(data.items()), columns=['ID', 'JSON_String'])
    df_json['ID'] = df_json['ID'].astype(int)
    df_json['JSON_String'] = df_json['JSON_String'].str.replace(',na)', ',"na")')
    

    # Function to convert JSON string to dictionary
    def json_str_to_dict(json_str):
        try:
            json_str = json_str.replace('output:', '')
            return ast.literal_eval(json_str)
        except:
            return "error:incorrect_format"

    # Convert JSON strings to dictionaries
    df_json['JSON_Dict'] = df_json['JSON_String'].apply(json_str_to_dict)

    # Convert dictionaries to DataFrame
    df = pd.json_normalize(df_json['JSON_Dict'])

    # Concatenate df_json and df
    df_extacted = pd.concat([df_json, df], axis=1)
    
    print('Extraction complete')
else:
    print("extraction_progress.json file does not exist.")

Extraction complete


In [107]:
df_extacted

,ID,JSON_String,JSON_Dict,time_of_day,reason,vehicles
0,107938160,"{'time_of_day': 'NA', 'reason': 'Negligence', ...","{'time_of_day': 'NA', 'reason': 'Negligence', ...",NA,Negligence,[Car]
1,107821301,"{'time_of_day': 'night', 'reason': 'Collision'...","{'time_of_day': 'night', 'reason': 'Collision'...",night,Collision,"[Car, Commercial Vehicle]"
2,107790964,"{'time_of_day': 'morning', 'reason': 'External...","{'time_of_day': 'morning', 'reason': 'External...",morning,External influences,"[Car, Commercial Vehicle]"
3,107737101,"{'time_of_day': 'morning', 'reason': 'Speed', ...","{'time_of_day': 'morning', 'reason': 'Speed', ...",morning,Speed,[Car]
4,107897709,"{'time_of_day': 'night', 'reason': 'External i...","{'time_of_day': 'night', 'reason': 'External i...",night,External influences,[Car]
...,...,...,...,...,...,...
8740,72050326,"{'time_of_day': 'NA', 'reason': 'NA', 'vehicle...","{'time_of_day': 'NA', 'reason': 'NA', 'vehicle...",NA,NA,"[Car, Special Vehicle]"
8741,72029018,"{'time_of_day': 'afternoon', 'reason': 'Extern...","{'time_of_day': 'afternoon', 'reason': 'Extern...",afternoon,External influences,[Car]
8742,56742830,"{'time_of_day': 'night', 'reason': 'Driving Un...","{'time_of_day': 'night', 'reason': 'Driving Un...",night,Driving Under Influence,"[Two-Wheeler, Commercial Vehicle]"
8743,67996026,"{'time_of_day': 'NA', 'reason': 'NA', 'vehicle...","{'time_of_day': 'NA', 'reason': 'NA', 'vehicle...",NA,NA,"[Car, Commercial Vehicle]"


In [109]:
df

,time_of_day,reason,vehicles
0,NA,Negligence,[Car]
1,night,Collision,"[Car, Commercial Vehicle]"
2,morning,External influences,"[Car, Commercial Vehicle]"
3,morning,Speed,[Car]
4,night,External influences,[Car]
...,...,...,...
8740,NA,NA,"[Car, Special Vehicle]"
8741,afternoon,External influences,[Car]
8742,night,Driving Under Influence,"[Two-Wheeler, Commercial Vehicle]"
8743,NA,NA,"[Car, Commercial Vehicle]"


In [108]:
df_extacted.to_csv('reasontimevehicle_classification.csv')

In [17]:
print(df_extacted.JSON_String[0])


{'time_of_day': NA, 'reason': 'Negligence', 'vehicles': ['Car']}
error:incorrect_format


In [18]:

print(df_extacted.JSON_Dict[0])

error:incorrect_format


# Backup

In [ ]:
class MessageInitializer:
    def __init__(self,option):
        if option == 1:
                self.message_history = [
            {
              "role": "system",
              "content":'''You will extract information from a news article and give me the output in json format.'''
            },
            {
              "role": "user",
              "content": "'New Delhi: Two students were injured after a speeding car rammed into the footpath they were walking on in central Delhi’s Ranjit Nagar. The errant driver has been booked, police said. The accident took place in the early hours of Tuesday and the students, in their early 20s, were discharged after treatment, a police officer said. “The driver who received some injuries was also rushed to the hospital. A case under sections 279 (rash driving), 337 (hurt due to negligence) and 338 (Punishment for grievous hurt) has been registered,” said the officer. The driver has been identified and the offending vehicle has been seized, police said. The purported CCTV footage of the incident started making rounds on social media on Thursday. In the video, a man and a woman can be seen walking on the footpath when a speeding car banged into the duo walking on the footpath. Explore Your Financial Landscape with Personalized Credit Insights.'"
            },
            {
              "role": "assistant",
              "content": "output:{\"fatalities\": 0,\"injured\":2,\"victim_gender\":[\"Male\",\"Female\"],\"vehicles\":[\"truck\",\"car\"],\"child_involved\":0,\"names_ages\":[(\"unknown_name\",20),(\"unknown_name\",20)]}/"
            }
          ]
        if option == 2:
                self.message_history = [
        {
          "role": "system",
          "content": "Input is a dict type.\n\nFor key value \"time_of_day\" classify and replace individual value to the following categories ['morning','afternoon','evening','night']\n\nFor key value \"reason\" classify and replace individual value to the following categories ['Speed','Driving Under Influence','Collision','Visibility','Negligence','External influences']\n\nFor key value \"vehicles\" classify and replace individual value to the following categories ['Car','Two-Wheeler','Commercial Vehicle','Special Vehicle']"
        },
        {
          "role": "user",
          "content": "'{\"time_of_day\":\"early morning\",\"reason\":\"rash driving\",\"vehicles\":[\"car\",\"tractor-trolly\"]}'"
        },
        {
          "role": "assistant",
          "content": "'{\"time_of_day\":\"morning\",\"reason\":\"Negligence\",\"vehicles\":[\"car\",\"'Special Vehicle'\"]}'"
        },
        {
          "role": "user",
          "content": "'{\"time_of_day\":\"early morning\",\"reason\":\"speeding\",\"vehicles\":[\"car\"]}'"
        },
        {
          "role": "assistant",
          "content": "'{\"time_of_day\":\"morning\",\"reason\":\"Speed\",\"vehicles\":[\"Car\"]}'"
        },
        {
          "role": "user",
          "content": "'{\"time_of_day\":\"wee hours\",\"reason\":\"Lost control over vehicle\",\"vehicles\":[\"BMW\",\"Truck\",\"scooty\"]}'"
        },
        {
          "role": "assistant",
          "content": "'{\"time_of_day\":\"night\",\"reason\":\"Negligence\",\"vehicles\":[\"Car\",\"Commercial Vehicle\",\"Two-Wheeler\"]}'"
        },
        {
          "role": "user",
          "content": "'{\"time_of_day\":\"night\",\"reason\":\"snorting drugs\",\"vehicles\":[\"Car\",\"Commercial Vehicle\",\"Two-Wheeler\"]}'"
        }
  ]
    
    def add_user_input(self, role, content):
        self.user_input = {"role": role, "content": content}
        self.prompt = self.message_history
        self.prompt.append(self.user_input)
        return (self.prompt)

In [ ]:
def process_message(data):
    
    m = MessageInitializer(2)
    message = m.add_user_input("user",df['JSON_Dict_filtered'][991])

    response = client.chat.completions.create(
                model="gpt-3.5-turbo-0125",
                messages = message,
                temperature=0.5,
                max_tokens=256,
                top_p=0.5,
                frequency_penalty=0,
                presence_penalty=0,
                stop = '/'
                                                )

    return response.choices[0].message.content